# Using Pyknow to design an expert system of RockPaperSicssor game.

In [47]:
#import the library
from pyknow import KnowledgeEngine
from pyknow import Rule
from pyknow import Fact
from pyknow import DefFacts
from pyknow import W # Wildcard Field Constraint
from pyknow import NOT # matching the absence of a given Fact/Pattern
from pyknow import Field
from pyknow import MATCH
from pyknow import AS
import random

        
class Results(Fact):
    winner = Field(str, mandatory=True)
    loser = Field(str, mandatory=True)
    why = Field(str, mandatory=True)
    
class ScoreBoard(Fact):
    win = Field(int, default=0)
    lose = Field(int, default=0)
    tie = Field(int, default=0)
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

class ValidAnswer(Fact):
    answer = Field(str, mandatory=True)
    key = Field(str, mandatory=True)
    
class Action(Fact):
    pass

class PlayerChoice(Action):
    key = Field(str, mandatory=True)


class Move(Action):
    role = Field(str, mandatory=True)
    answer = Field(str, mandatory=True)


class Quit(Action):
    quit = Field(str, mandatory=True)


def logger(func):
    def inner(*args, **kwargs):
        print(f'calling {func.__name__} with args={args} and kwargs={kwargs}')
        return func(*args, **kwargs)
    return inner


class RockPaperScissorsGame(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        self.valid_answers = dict()
        
        # make the rules
        yield ScoreBoard(win=0, lose=0, tie=0)
        yield Results(winner='rock', loser='scissors', why='Rock smashes scissors')
        yield Results(winner='paper', loser='rock', why='Paper covers rock')
        yield Results(winner='scissors', loser='paper', why='Scissors cut paper')
        
        #regulate the input
        yield ValidAnswer(answer='rock', key='r')
        yield ValidAnswer(answer='paper', key='p')
        yield ValidAnswer(answer='scissors', key='s')
        
    @Rule(NOT(Action()),ValidAnswer(answer=MATCH.answer, key=MATCH.key))
    @logger
    def store_valid_answers(self, answer, key):
        self.valid_answers[key] = answer
        
    @Rule()
    @logger
    def startup(self):
        self.declare(Action('get-move'))
        print("Lets play rock-paper-scissors!")
        self.declare(Action('get-player-move'))
        
    #player_choice    
    @Rule(AS.event << Action('get-player-move'))
    def get_player_move(self, event):
        self.retract(event)
        play_choice = input('please input your choice (R) for Rock, (S) for Scissor, (P) for Paper\n')
        choice = play_choice.lower()
        self.declare(PlayerChoice(key=choice))
        
    @Rule(AS.event << PlayerChoice(key=MATCH.choice),
      ValidAnswer(key=MATCH.choice, answer=MATCH.answer))
    def triggle_computer(self, event, answer):
        self.retract(event)
        self.declare(Move(answer=answer, role='player'))
        
    @Rule(AS.event << PlayerChoice(key=MATCH._choice),
          NOT(ValidAnswer(key=MATCH._choice)))
    def player_unvalide_move(self, event):
        self.retract(event)
        print("Wrong input,try again")
        self.declare(Action('get-player-move'))
        
        
    #computer_choice   
    @Rule(Move(role='player'))
    def get_computer_action(self):
        choice = random.choice(list(self.valid_answers))
        self.declare(Move(answer=self.valid_answers[choice], role='computer'))
        
    #print the choice made by computer and user
    @Rule(AS.e2 << Move(answer=MATCH.player_answer, role='player'),
          AS.e3 << Move(answer=MATCH.computer_answer, role='computer'))
    def print_both_answers(self, player_answer, computer_answer):
        print("*************************")
        print(
            f'You chose:{player_answer}, and computer chose: {computer_answer}')
        self.declare(Action('Judge'))
        
        
    #judge the result
    @Rule(AS.e1 << Action('Judge'),
          AS.e2 << Move(answer=MATCH.player_answer, role='player'),
          AS.e3 << Move(answer=MATCH.computer_answer, role='computer'),
          AS.board << ScoreBoard(win=MATCH.wins),
          Results(winner=MATCH.player_answer,
                  loser=MATCH.computer_answer,
                  why=MATCH.explaination))
    def player_win(self, e1, e2, e3, board, explaination, wins):
        self.retract(e1)
        self.retract(e2)
        self.retract(e3)

        self.modify(board, win=wins + 1)
        print(f'You win! {explaination}')
        self.declare(Action('game-over'))
        
        
    @Rule(AS.e1 << Action('Judge'),
          AS.e2 << Move(answer=MATCH.player_answer, role='player'),
          AS.e3 << Move(answer=MATCH.computer_answer, role='computer'),
          AS.board << ScoreBoard(lose=MATCH.loses),
          Results(winner=MATCH.computer_answer,
                  loser=MATCH.player_answer,
                  why=MATCH.explaination))
    def computer_win(self, e1, e2, e3, board, explaination, loses):
        self.retract(e1)
        self.retract(e2)
        self.retract(e3)

        print(f'Computer wins! {explaination}')
        self.modify(board, lose=loses + 1)
        self.declare(Action('game-over'))
    
    @Rule(AS.e1 << Action('Judge'),
          AS.e2 << Move(answer=MATCH.player_answer, role='player'),
          AS.e3 << Move(answer=MATCH.computer_answer, role='computer'),
          NOT(Results(winner=MATCH.computer_answer,
                  loser=MATCH.player_answer)),
          NOT(Results(winner=MATCH.computer_answer,
                  loser=MATCH.computer_answer)),                  
          AS.board << ScoreBoard(tie=MATCH.ties))
    def tie(self, e1, e2, e3, board, ties):
        self.retract(e1)
        self.retract(e2)
        self.retract(e3)
        print('tie')
        self.modify(board, tie=ties + 1)
        self.declare(Action('game-over'))
        
    #print the total result, ask will continue game or not 
    @Rule(AS.e << Action('game-over'),
          AS.board << ScoreBoard(lose=MATCH.loses, win=MATCH.wins, tie=MATCH.ties))
    def game_over(self, e, board, loses, wins, ties):
        self.retract(e)
        total = loses + wins + ties
        print(f'You have played {total} times, {wins} wins, {loses} loses and {ties} ties.')
        print("*************************")
     
        choice = input("Would you like to play again?  Input N to leave, otherwise continue")
        self.declare(Quit(quit=choice.lower()))
      
    #quit the game
    @Rule(AS.e << Quit(quit='n'))
    def quit(self, e):
        self.retract(e)
        print("Good bye!")
        self.halt()

    #continue to play
    @Rule(AS.e << Quit(),
          NOT(Quit(quit='n')))
    def play_again(self, e):
        self.retract(e)
        self.declare(Action('get-player-move'))

In [48]:
game = RockPaperScissorsGame()
game.reset()
game.run()

calling store_valid_answers with args=(<__main__.RockPaperScissorsGame object at 0x00000267A3236F70>,) and kwargs={'answer': 'scissors', 'key': 's'}
calling store_valid_answers with args=(<__main__.RockPaperScissorsGame object at 0x00000267A3236F70>,) and kwargs={'key': 'p', 'answer': 'paper'}
calling store_valid_answers with args=(<__main__.RockPaperScissorsGame object at 0x00000267A3236F70>,) and kwargs={'answer': 'rock', 'key': 'r'}
calling startup with args=(<__main__.RockPaperScissorsGame object at 0x00000267A3236F70>,) and kwargs={}
Lets play rock-paper-scissors!
please input your choice (R) for Rock, (S) for Scissor, (P) for Paper
r
*************************
You chose:rock, and computer chose: rock
tie
You have played 1 times, 0 wins, 0 loses and 1 ties.
**********************
Would you like to play again?  Input N to leave, otherwise continuew
please input your choice (R) for Rock, (S) for Scissor, (P) for Paper
p
*************************
You chose:paper, and computer chose: p